In [2]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')


leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)




/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:55: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_19354/2775728346.py:62: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [3]:
#QUERY GOOGLE FUNCTIONS

#FUNCTIONS

def find_transfermarkt_links(html_content):
    # Regular expression patterns for transfermarkt URLs
    transfermarkt_patterns = [
        r"transfermarkt\.de",
        r"transfermarkt\.us",
        r"transfermarkt\.world"
    ]
    transfermarkt_links = []

    # Find all 'a' tags in the HTML content
    links = html_content.find_all('a', href=True)

    # Filter links that match transfermarkt patterns
    for link in links:
        for pattern in transfermarkt_patterns:
            if re.search(pattern, link['href']):
                transfermarkt_links.append(link['href'])
                break

    return transfermarkt_links


def query_google_1(input_string):
    # Replace spaces with plus signs to format the query string
    query = '+'.join(input_string.split())

    # URL for Google search
    url = f"https://www.google.com/search?q={query}"

    # Send GET request to Google
    response = requests.get(url)

    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Return the parsed HTML content
        return soup
    else:
        # If request fails, print error message
        print("Failed to fetch results from Google.")
        return None
    

def extract_name_from_link(link):
        
    name_in_link = link.split('/')[3]
    name_in_link_tokens = name_in_link.split('-')
    capitalized_tokens = []

    for token in name_in_link_tokens:
        capitalized_tokens.append(token.capitalize())

    combined_string = ' '.join(capitalized_tokens)

    return combined_string


def process_google_query(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        transfermarkt_links = find_transfermarkt_links(html_result)
        if transfermarkt_links:
            names_links = []
            for link in transfermarkt_links:
                if link.startswith('/url?q='):
                    link = link.split('/url?q=')[1]
                name_from_link = extract_name_from_link(link)
                names_links.append(name_from_link)
            if len(names_links) == 1:
                name = names_links[0]
                return name
            elif len(names_links) >= 2:
                name_list_matches = list(set(names_links))
                return name_list_matches
            else:
                return 0  # Nobody found
        else:
            return 0  # No links found
    else:
        return 0  # No HTML result

In [4]:
def useSeleniumToFindMoney(
        NAME_TO_FIND, #the name it is looking up
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN):
    if(SALARY_BOOLEAN == False):
        print('finding salary with selenium')
        marketvalue = AssemblyHelpers.seleniumLookUpValueWrapperFunction(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return marketvalue
    elif(SALARY_BOOLEAN == True):
        salary = AssemblyHelpers.capology_selenium_lookup(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return salary

    else:
        return 0

### Adding manual fixes  

#### WC Europe

In [4]:
wcs_europe = load_csv_dataset('world_cup_quals_europe_lineups_complete.csv')

In [7]:
wcs_europe.at[16920, 'Market Value'] = 50000
wcs_europe.at[16922, 'Market Value'] = 50000
wcs_europe.at[16928, 'Market Value'] = 50000

In [10]:
# for index, row in wcs_europe.iterrows():
for i in range(16908, len(wcs_europe)):
    index = i
    row = wcs_europe.iloc[index]
    if row['Status'] != 'SUCCESS':
        name = row["Name(s) Found"]
        nationality = row['Nationality']
        date = row['Date']
        salary_boolean = False
        test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
        if test_figure == 0:
            wcs_europe.at[index, 'Market Value'] = test_figure
            wcs_europe.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            wcs_europe.at[index, 'Lookup Still Required?'] = True
            wcs_europe.at[index, 'Status'] = 'FAIL - ?'
        else:
            wcs_europe.at[index, 'Market Value'] = test_figure
            wcs_europe.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            wcs_europe.at[index, 'Lookup Still Required?'] = False
            wcs_europe.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

In [28]:
wcs_europe.to_csv('world_cup_quals_europe_lineups_complete_EDITED.csv')

#### WC ASIA

In [31]:
wcs_asia = load_csv_dataset('world_cup_qualifiers_asia_lineups_complete.csv')

Status
SUCCESS          13033
FAIL - EMPTY       116
FAIL - NEW          70
FAIL - MANUAL       69
Name: count, dtype: int64

In [50]:
wcs_asia.at[13251, 'Status'] = 'SUCCESS'
wcs_asia.at[13251, 'Market Value'] = 100000
wcs_asia.at[13251, 'Lookup Still Required?'] = False
wcs_asia.at[13251, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13076, 'Status'] = 'SUCCESS'
wcs_asia.at[13076, 'Market Value'] = 0
wcs_asia.at[13076, 'Lookup Still Required?'] = False
wcs_asia.at[13076, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13188, 'Status'] = 'SUCCESS'
wcs_asia.at[13188, 'Market Value'] = 0
wcs_asia.at[13188, 'Lookup Still Required?'] = False
wcs_asia.at[13188, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13190, 'Status'] = 'SUCCESS'
wcs_asia.at[13190, 'Market Value'] = 0
wcs_asia.at[13190, 'Lookup Still Required?'] = False
wcs_asia.at[13190, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13247, 'Status'] = 'SUCCESS'
wcs_asia.at[13247, 'Market Value'] = 0
wcs_asia.at[13247, 'Lookup Still Required?'] = False
wcs_asia.at[13247, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13248, 'Status'] = 'SUCCESS'
wcs_asia.at[13248, 'Market Value'] = 0
wcs_asia.at[13248, 'Lookup Still Required?'] = False
wcs_asia.at[13248, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13264, 'Status'] = 'SUCCESS'
wcs_asia.at[13264, 'Market Value'] = 0
wcs_asia.at[13264, 'Lookup Still Required?'] = False
wcs_asia.at[13264, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13287, 'Status'] = 'SUCCESS'
wcs_asia.at[13287, 'Market Value'] = 0
wcs_asia.at[13287, 'Lookup Still Required?'] = False
wcs_asia.at[13287, 'Lookup Return Case'] = 'Found in Curr Season'

In [107]:
for index, row in wcs_asia.iterrows():
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
    if row['Status'] != 'SUCCESS':
        if row['Status'] == 'FAIL - MANUAL':
            name = row["Name(s) Found"]
            nationality = row['Nationality']
            date = row['Date']
            salary_boolean = False
            test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
            if test_figure == 0:
                wcs_asia.at[index, 'Market Value'] = test_figure
                wcs_asia.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                wcs_asia.at[index, 'Lookup Still Required?'] = True
                wcs_asia.at[index, 'Status'] = 'FAIL - ?'
            else:
                wcs_asia.at[index, 'Market Value'] = test_figure
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is not valid.
Ahmad Salem Mohamad https://www.transfermarkt.us/ahmad-salem-mohamad/nationalmannschaft/spieler/372589
Mohamad Salem https://www.transfermarkt.us/mohamad-salem/nationalmannschaft/spieler/1107627
Mohammed Salem https://www.transfermarkt.us/mohammed-salem/nationalmannschaft/spieler/185640
0 matches []
husain ali pele case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://ww

In [60]:
wcs_asia.at[13033, 'Status'] = 'SUCCESS'
wcs_asia.at[13033, 'Market Value'] = 1000000
wcs_asia.at[13033, 'Lookup Still Required?'] = False
wcs_asia.at[13033, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13037, 'Status'] = 'SUCCESS'
wcs_asia.at[13037, 'Market Value'] = 700000
wcs_asia.at[13037, 'Lookup Still Required?'] = False
wcs_asia.at[13037, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13034, 'Status'] = 'SUCCESS'
wcs_asia.at[13034, 'Market Value'] = 700000
wcs_asia.at[13034, 'Lookup Still Required?'] = False
wcs_asia.at[13034, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13036, 'Status'] = 'SUCCESS'
wcs_asia.at[13036, 'Market Value'] = 700000
wcs_asia.at[13036, 'Lookup Still Required?'] = False
wcs_asia.at[13036, 'Lookup Return Case'] = 'Found in Curr Season'

In [76]:
wcs_asia.at[13058, 'Status'] = 'SUCCESS'
wcs_asia.at[13058, 'Market Value'] = 150000
wcs_asia.at[13058, 'Lookup Still Required?'] = False
wcs_asia.at[13058, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13060, 'Status'] = 'SUCCESS'
wcs_asia.at[13060, 'Market Value'] = 150000
wcs_asia.at[13060, 'Lookup Still Required?'] = False
wcs_asia.at[13060, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13061, 'Status'] = 'SUCCESS'
wcs_asia.at[13061, 'Market Value'] = 150000
wcs_asia.at[13061, 'Lookup Still Required?'] = False
wcs_asia.at[13061, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13062, 'Status'] = 'SUCCESS'
wcs_asia.at[13062, 'Market Value'] = 150000
wcs_asia.at[13062, 'Lookup Still Required?'] = False
wcs_asia.at[13062, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13064, 'Status'] = 'SUCCESS'
wcs_asia.at[13064, 'Market Value'] = 150000
wcs_asia.at[13064, 'Lookup Still Required?'] = False
wcs_asia.at[13064, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13057, 'Status'] = 'SUCCESS'
wcs_asia.at[13057, 'Market Value'] = 75000
wcs_asia.at[13057, 'Lookup Still Required?'] = False
wcs_asia.at[13057, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13066, 'Status'] = 'SUCCESS'
wcs_asia.at[13066, 'Market Value'] = 150000
wcs_asia.at[13066, 'Lookup Still Required?'] = False
wcs_asia.at[13066, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13059, 'Status'] = 'SUCCESS'
wcs_asia.at[13059, 'Market Value'] = 300000
wcs_asia.at[13059, 'Lookup Still Required?'] = False
wcs_asia.at[13059, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13063, 'Status'] = 'SUCCESS'
wcs_asia.at[13063, 'Market Value'] = 300000
wcs_asia.at[13063, 'Lookup Still Required?'] = False
wcs_asia.at[13063, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13065, 'Status'] = 'SUCCESS'
wcs_asia.at[13065, 'Market Value'] = 300000
wcs_asia.at[13065, 'Lookup Still Required?'] = False
wcs_asia.at[13065, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13067, 'Status'] = 'SUCCESS'
wcs_asia.at[13067, 'Market Value'] = 300000
wcs_asia.at[13067, 'Lookup Still Required?'] = False
wcs_asia.at[13067, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13070, 'Status'] = 'SUCCESS'
wcs_asia.at[13070, 'Market Value'] = 50000
wcs_asia.at[13070, 'Lookup Still Required?'] = False
wcs_asia.at[13070, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13073, 'Status'] = 'SUCCESS'
wcs_asia.at[13073, 'Market Value'] = 70000
wcs_asia.at[13073, 'Lookup Still Required?'] = False
wcs_asia.at[13073, 'Lookup Return Case'] = 'Found in Curr Season'

In [83]:
#1 no 2 no 3 no 4 no 5 no 6 no 
for index, row in wcs_asia.iterrows():
    if row['Status'] == 'FAIL - ?':
        wcs_asia.at[index, 'Status'] = 'SUCCESS'
        wcs_asia.at[index, 'Market Value'] = 0
        wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'

In [117]:
for index, row in wcs_asia.iterrows():
    if row['Status'] == 'FAIL - ?':
        if row['Name(s) Found'] == 'Mohanad Salem':
            if row['Season'] == 2016:
                0==0
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
                wcs_asia.at[index, 'Market Value'] = 200000
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            elif row['Season'] == 2017:
                0==0
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
                wcs_asia.at[index, 'Market Value'] = 225000
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            else:
                0==0
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
                wcs_asia.at[index, 'Market Value'] = 25000
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                
            print(index, row['Season'])

In [123]:
wcs_asia.at[13161, 'Status'] = 'SUCCESS'
wcs_asia.at[13161, 'Market Value'] = 300000
wcs_asia.at[13161, 'Lookup Still Required?'] = False
wcs_asia.at[13161, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13246, 'Status'] = 'SUCCESS'
wcs_asia.at[13246, 'Market Value'] = 25000
wcs_asia.at[13246, 'Lookup Still Required?'] = False
wcs_asia.at[13246, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13189, 'Status'] = 'SUCCESS'
wcs_asia.at[13189, 'Market Value'] = 0
wcs_asia.at[13189, 'Lookup Still Required?'] = False
wcs_asia.at[13189, 'Lookup Return Case'] = 'Found in Curr Season'

In [127]:
wcs_asia.to_csv('world_cup_qualifiers_asia_lineups_complete_EDITED.csv')

#### WCQ Africa

In [128]:
wcs_africa = load_csv_dataset('world_cup_qualifiers_africa_lineups_complete.csv')

In [182]:
wcs_africa['Status'].value_counts()

Status
SUCCESS           9373
FAIL - NEW         171
FAIL - NO DATA      66
FAIL - EMPTY         4
Name: count, dtype: int64

In [189]:
for index, row in wcs_africa.iterrows():
    if index >= 9274:
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - NEW':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    wcs_africa.at[index, 'Market Value'] = test_figure
                    wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    wcs_africa.at[index, 'Lookup Still Required?'] = False
                    wcs_africa.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/patrick-gerhardt-nyema/marktwertverlauf/spieler/56275
passed 1
passed 2
passed 3
{'value': '2007', 'coordinates': (67.55325610167586, 0.0)} 07
{'value': '2009', 'coordinates': (180.02744348217297, 0.0)} 09
{'value': '2011', 'coordinates': (292.34776740524126, 0.0)} 11
{'value': '2013', 'coordinates': (404.8219547857384, 0.0)} 13
{'value': '2015', 'coordinates': (517.1422787088067, 0.0)} 15
{'value': '2017', 'coordinates': (629.6164660893038, 0.0)} 17
passed 4
passed 5
passed 6
500000.0 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/hugo-marques/ma

In [179]:
for index, row in wcs_africa.iterrows():
    if row['Status'] == 'FAIL - ?':
        if row['Name(s) Found'] == 'Nadir Eltayeb':
            if row['Season'] == 2016:
                0==0
                # wcs_africa.at[index, 'Status'] = 'SUCCESS'
                # wcs_africa.at[index, 'Market Value'] = 0
                # wcs_africa.at[index, 'Lookup Still Required?'] = False
                # wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            elif row['Season'] == 2017:
                0==0
                # wcs_africa.at[index, 'Status'] = 'SUCCESS'
                # wcs_africa.at[index, 'Market Value'] = 0
                # wcs_africa.at[index, 'Lookup Still Required?'] = False
                # wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            else:
                0==0
                wcs_africa.at[index, 'Status'] = 'SUCCESS'
                wcs_africa.at[index, 'Market Value'] = 0
                wcs_africa.at[index, 'Lookup Still Required?'] = False
                wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                
            print(index, row['Season'])

9163 2013


In [193]:
wcs_africa.to_csv('world_cup_qualifiers_africa_lineups_complete_EDITED.csv')

In [201]:
wcs_asia['Status'].value_counts()

Status
SUCCESS         13134
FAIL - EMPTY      116
FAIL - NEW         38
Name: count, dtype: int64

In [199]:
for index, row in wcs_asia.iterrows():
    if index >= 13227:
#for i in range(13033, 13288):
#index = i
#row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - NEW':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    wcs_asia.at[index, 'Market Value'] = test_figure
                    wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    wcs_asia.at[index, 'Lookup Still Required?'] = False
                    wcs_asia.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
    #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/aloisio/marktwertverlauf/spieler/55246
passed 1
passed 2
passed 3
{'value': '2009', 'coordinates': (120.88471085176799, 0.0)} 09
{'value': '2011', 'coordinates': (200.61123933112697, 0.0)} 11
{'value': '2013', 'coordinates': (280.4469822330604, 0.0)} 13
{'value': '2015', 'coordinates': (360.17351071241944, 0.0)} 15
{'value': '2017', 'coordinates': (440.00925361435293, 0.0)} 17
{'value': '2019', 'coordinates': (519.7357820937119, 0.0)} 19
{'value': '2021', 'coordinates': (599.5715249956454, 0.0)} 21
passed 4
passed 5
passed 6
200000.0 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/aloisio/marktwertverlauf/spieler/55246
passed 1
passed 2
passed 3
{'value': '2009', 'coordinates': (120.88471085176799, 0.0)} 09
{'value': '2011', 'coordinates': (200.61123933112697, 0.0)} 11
{'value': '2013', 'coordinates': (28

### AFCON Qualifiers

In [4]:
new_afcon_qualis_data = load_csv_dataset('afcon_qualis_lineups_complete.csv')

In [19]:
new_afcon_qualis_data['Status'].value_counts()

Status
SUCCESS          15556
FAIL - EMPTY       169
FAIL - MANUAL        5
Name: count, dtype: int64

In [7]:
salary_boolean = False

In [14]:
for index, row in new_afcon_qualis_data.iterrows():
    if index >= 15624:
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - MANUAL':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == -1:
                    new_afcon_qualis_data.at[index, 'Market Value'] = 0
                    new_afcon_qualis_data.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    new_afcon_qualis_data.at[index, 'Lookup Still Required?'] = False
                    new_afcon_qualis_data.at[index, 'Status'] = 'SUCCESS'
                elif test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    new_afcon_qualis_data.at[index, 'Market Value'] = test_figure
                    new_afcon_qualis_data.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    new_afcon_qualis_data.at[index, 'Lookup Still Required?'] = False
                    new_afcon_qualis_data.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/salim-magoola/marktwertverlauf/spieler/315816
passed 1
passed 2
passed 3
{'value': '2015', 'coordinates': (87.49122360722463, 0.0)} 15
{'value': '2017', 'coordinates': (234.76977868226913, 0.0)} 17
{'value': '2019', 'coordinates': (381.8468583057746, 0.0)} 19
{'value': '2021', 'coordinates': (529.125413380819, 0.0)} 21
{'value': '2023', 'coordinates': (676.2024930043245, 0.0)} 23
passed 4
passed 5
passed 6
650000.0 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/abdelkahar-kadri/marktwertverlauf/spieler/592400
passed 1
passed 2
passed 3
{'value': '2019', 'coordinates': (100.5899314832941, 0.0)} 19
{'value': '2020', 'coordinates': (246.2889341551811, 0.0)} 20
{'value': '2021', 'coordinates': (391.5898521203143, 0.0)} 21
{'value': '2022', 'coordinates': (536.8907700854475, 0.0)} 22
passed 4
passed 5
passed 

In [22]:
for index, row in new_afcon_qualis_data.iterrows():
    if row['Market Value'] < 0:
        new_afcon_qualis_data.at[index, 'Market Value'] = 0
    if row['Status'] == 'FAIL - MANUAL':
        new_afcon_qualis_data.at[index, 'Market Value'] = 0
        new_afcon_qualis_data.at[index, 'Status'] = 'SUCCESS'
        new_afcon_qualis_data.at[index, 'Lookup Return Case'] = 'Found in Curr Season'

In [28]:
new_afcon_qualis_data.to_csv('afcon_qualis_lineups_complete_EDITED.csv')

### Asia WCQs 2026

In [31]:
asia_wcqs_2026 = load_csv_dataset('asia_world_cup_qualis_2026.csv')

In [35]:
for index, row in asia_wcqs_2026.iterrows():
    if index >= 0:
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - MANUAL':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == -1:
                    asia_wcqs_2026.at[index, 'Market Value'] = 0
                    asia_wcqs_2026.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    asia_wcqs_2026.at[index, 'Lookup Still Required?'] = False
                    asia_wcqs_2026.at[index, 'Status'] = 'SUCCESS'
                elif test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    asia_wcqs_2026.at[index, 'Market Value'] = test_figure
                    asia_wcqs_2026.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    asia_wcqs_2026.at[index, 'Lookup Still Required?'] = False
                    asia_wcqs_2026.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
Retrying...
Loading... message detected
https://www.transfermarkt.us/mohammed-osman/marktwertverlauf/spieler/273431
passed 1
passed 2
passed 3
{'value': '2015', 'coordinates': (86.50181185679084, 0.0)} 15
{'value': '2017', 'coordinates': (231.45151471227712, 0.0)} 17
{'value': '2019', 'coordinates': (376.202927960574, 0.0)} 19
{'value': '2021', 'coordinates': (521.1526308160603, 0.0)} 21
{'value': '2023', 'coordinates': (665.9040440643572, 0.0)} 23
passed 4
passed 5
passed 6
349999.99999999977 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/mohammed-osman/marktwertverlauf/spie

In [38]:
asia_wcqs_2026.at[1209, 'Market Value'] = 200000
asia_wcqs_2026.at[1209, 'Status'] = 'SUCCESS'

asia_wcqs_2026.at[1212, 'Market Value'] = 25000
asia_wcqs_2026.at[1212, 'Status'] = 'SUCCESS'

asia_wcqs_2026.at[1222, 'Market Value'] = 0
asia_wcqs_2026.at[1222, 'Status'] = 'SUCCESS'

asia_wcqs_2026.at[1228, 'Market Value'] = 0
asia_wcqs_2026.at[1228, 'Status'] = 'SUCCESS'

asia_wcqs_2026.at[1226, 'Market Value'] = 0
asia_wcqs_2026.at[1226, 'Status'] = 'SUCCESS'

In [41]:
asia_wcqs_2026.to_csv('asia_world_cup_LINEUPS_EDITED.csv')

### checking data sizes

In [87]:
combined_df = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/combined_df_may_16_gold_cup_removed_UPDATED.csv')
#len(combined_df)
combined_df = combined_df.drop(columns={'Unnamed: 0'})
desired_order = [
    'Match ID', 'Competition', 'Match', 'Date', 'Nationality', 'Team Country Code',
    'Season', 'Status', 'Name', 'Market Value', 'Lookup Still Required?',
    'Lookup Return Case', 'Impute Required?', 'Name(s) Found', 'ORIGINAL JERSEY',
    'Match Case', 'Competition General'
]
combined_df = combined_df.reindex(columns=desired_order)

combined_df

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_24576/2775728346.py:54: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Mohammed Hameed,100000.0,False,working,False,Mohammed Hameed,Mohammed Hameed,single,Olympics
1,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Adnan,1800000.0,False,working,False,Ali Adnan,Ali Adnan,single,Olympics
2,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Dhurgham Ismail,1250000.0,False,working,False,Dhurgham Ismail,Dhurgham Ismail,single,Olympics
3,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Husni,100000.0,False,working,False,Ali Husni,Ali Hosni,single,Olympics
4,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Humam Tariq,100000.0,False,working,False,Humam Tariq,Humam Tariq,single,Olympics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50705,Nations League_443,Nations League 2022-2023,Andorra vs Liechtenstein,2022-06-10,Andorra,AD,2022,SUCCESS,Josep Gómes,10000.0,False,Found in Curr Season,False,Josep Gómes,1 Josep Gómes,single,Nations League
50706,Nations League_444,Nations League 2022-2023,Moldova vs Andorra,2022-06-14,Moldova,MD,2022,SUCCESS,Dorian Railean,200000.0,False,Found in Curr Season,False,Dorian Railean,23 Dorian Railean,single,Nations League
50707,World Cup Intercontinental Playoffs_2059,"World Cup Intercontinental Playoffs, 2018",Peru vs New Zealand,16-11-2017,Peru,PE,2017,SUCCESS,Alberto Rodríguez,2000000.0,False,Found in Curr Season,False,Alberto Rodríguez,A Rodríguez,single,World Cup Intercontinental Playoffs
50708,World Cup Intercontinental Playoffs_2062,"World Cup Intercontinental Playoffs, 2018",New Zealand vs Peru,11-11-2017,Peru,PE,2017,SUCCESS,Alberto Rodríguez,2000000.0,False,Found in Curr Season,False,Alberto Rodríguez,A Rodríguez,single,World Cup Intercontinental Playoffs


In [83]:
wcq_europe = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/world_cup_quals_europe_lineups_complete_EDITED.csv')
wcq_europe['Competition General'] = 'World Cup Qualifiers, Europe'
#len(wcq_europe)
wcq_europe = wcq_europe.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})
wcq_europe

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,"World Cup Qualifiers, Europe_0","World Cup Qualifiers, 2014, Europe",Croatia vs Macedonia,2012-09-07,Croatia,HR,2012,SUCCESS,Stipe Pletikosa,1800000.0,False,Found in Curr Season,False,Stipe Pletikosa,1 Stipe Pletikosa,single,"World Cup Qualifiers, Europe"
1,"World Cup Qualifiers, Europe_0","World Cup Qualifiers, 2014, Europe",Croatia vs Macedonia,2012-09-07,Croatia,HR,2012,SUCCESS,Josip Simunic,1000000.0,False,Found in Curr Season,False,Josip Simunic,3 Josip Simunic,single,"World Cup Qualifiers, Europe"
2,"World Cup Qualifiers, Europe_0","World Cup Qualifiers, 2014, Europe",Croatia vs Macedonia,2012-09-07,Croatia,HR,2012,SUCCESS,Vedran Corluka,8000000.0,False,Found in Curr Season,False,Vedran Corluka,5 Vedran Corluka,single,"World Cup Qualifiers, Europe"
3,"World Cup Qualifiers, Europe_0","World Cup Qualifiers, 2014, Europe",Croatia vs Macedonia,2012-09-07,Croatia,HR,2012,SUCCESS,Ivan Strinic,5500000.0,False,Found in Curr Season,False,Ivan Strinic,2 Ivan Strinic,single,"World Cup Qualifiers, Europe"
4,"World Cup Qualifiers, Europe_0","World Cup Qualifiers, 2014, Europe",Croatia vs Macedonia,2012-09-07,Croatia,HR,2012,SUCCESS,Darijo Srna,10000000.0,False,Found in Curr Season,False,Darijo Srna,11 Darijo Srna,single,"World Cup Qualifiers, Europe"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17155,"World Cup Qualifiers, Europe_775","World Cup Qualifiers, 2022, Europe",Romania vs Iceland,2021-11-11,Iceland,IS,2021,SUCCESS,Sveinn Aron Gudjohnsen,350000.0,False,Found in Curr Season,False,Sveinn Aron Gudjohnsen,19 Sveinn Aron Gudjohnsen,single,"World Cup Qualifiers, Europe"
17156,"World Cup Qualifiers, Europe_776","World Cup Qualifiers, 2022, Europe",Germany vs Liechtenstein,2021-11-11,Liechtenstein,LI,2021,SUCCESS,Dennis Salanovic,250000.0,False,Found in Curr Season,False,Dennis Salanovic,11 Dennis Salanovic,single,"World Cup Qualifiers, Europe"
17157,"World Cup Qualifiers, Europe_777","World Cup Qualifiers, 2022, Europe",Liechtenstein vs Romania,2021-11-14,Romania,RO,2021,SUCCESS,Olimpiu Moruțan,5000000.0,False,Found in Curr Season,False,Olimpiu Moruțan,21 Olimpiu Moruțan,single,"World Cup Qualifiers, Europe"
17158,"World Cup Qualifiers, Europe_779","World Cup Qualifiers, 2022, Europe",North Macedonia vs Iceland,2021-11-14,Iceland,IS,2021,SUCCESS,Ísak Jóhannesson,4500000.0,False,Found in Curr Season,False,Ísak Jóhannesson,7 Ísak Jóhannesson,single,"World Cup Qualifiers, Europe"


In [82]:
wcq_asia = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/world_cup_qualifiers_asia_lineups_complete_EDITED.csv')
wcq_asia['Competition General'] = 'World Cup Qualifiers, Asia'
#len(wcq_asia)
wcq_asia = wcq_asia.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})
wcq_asia

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,"World Cup Qualifiers, Asia_0","World Cup Qualifiers, 2014, Asia",Uzbekistan vs Jordan,2013-09-10,Uzbekistan,UZ,2013,SUCCESS,Murod Zukhurov,75000.0,False,Found in Curr Season,False,Murod Zukhurov,- Murod Zukhurov,single,"World Cup Qualifiers, Asia"
1,"World Cup Qualifiers, Asia_0","World Cup Qualifiers, 2014, Asia",Uzbekistan vs Jordan,2013-09-10,Uzbekistan,UZ,2013,SUCCESS,Anzur Ismoilov,200000.0,False,Found in Curr Season,False,Anzur Ismoilov,- Anzur Ismoilov,single,"World Cup Qualifiers, Asia"
2,"World Cup Qualifiers, Asia_0","World Cup Qualifiers, 2014, Asia",Uzbekistan vs Jordan,2013-09-10,Uzbekistan,UZ,2013,SUCCESS,Islom Tukhtakhuzhaev,350000.0,False,Found in Curr Season,False,Islom Tukhtakhuzhaev,- Islom Tukhtakhuzhaev,single,"World Cup Qualifiers, Asia"
3,"World Cup Qualifiers, Asia_0","World Cup Qualifiers, 2014, Asia",Uzbekistan vs Jordan,2013-09-10,Uzbekistan,UZ,2013,SUCCESS,Vitaliy Denisov,3500000.0,False,Found in Curr Season,False,Vitaliy Denisov,- Vitaliy Denisov,single,"World Cup Qualifiers, Asia"
4,"World Cup Qualifiers, Asia_0","World Cup Qualifiers, 2014, Asia",Uzbekistan vs Jordan,2013-09-10,Uzbekistan,UZ,2013,SUCCESS,Oleg Zoteev,275000.0,False,Used Average of Season Before and After,False,Oleg Zoteev,- Oleg Zoteev,single,"World Cup Qualifiers, Asia"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13283,"World Cup Qualifiers, Asia_597","World Cup Qualifiers, 2022, Asia",Sri Lanka vs Macau,2019-06-27,Macau,MF,2019,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Asia"
13284,"World Cup Qualifiers, Asia_597","World Cup Qualifiers, 2022, Asia",Sri Lanka vs Macau,2019-06-27,Macau,MF,2019,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Asia"
13285,"World Cup Qualifiers, Asia_597","World Cup Qualifiers, 2022, Asia",Sri Lanka vs Macau,2019-06-27,Macau,MF,2019,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Asia"
13286,"World Cup Qualifiers, Asia_597","World Cup Qualifiers, 2022, Asia",Sri Lanka vs Macau,2019-06-27,Macau,MF,2019,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Asia"


In [80]:
wcq_africa = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/world_cup_qualifiers_africa_lineups_complete_EDITED.csv')
wcq_africa['Competition General'] = 'World Cup Qualifiers, Africa'
#len(wcq_africa)
wcq_africa = wcq_africa.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})
wcq_africa

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,"World Cup Qualifiers, Africa_0","World Cup Qualifiers, 2014, Africa",Nigeria vs Ethiopia,2013-11-16,Nigeria,NG,2013,SUCCESS,Vincent Enyeama,1500000.0,False,Found in Curr Season,False,Vincent Enyeama,1 Vincent Enyeama,single,"World Cup Qualifiers, Africa"
1,"World Cup Qualifiers, Africa_0","World Cup Qualifiers, 2014, Africa",Nigeria vs Ethiopia,2013-11-16,Nigeria,NG,2013,SUCCESS,Godfrey Oboabona,300000.0,False,Found in Curr Season,False,Godfrey Oboabona,2 Godfrey Oboabona,single,"World Cup Qualifiers, Africa"
2,"World Cup Qualifiers, Africa_0","World Cup Qualifiers, 2014, Africa",Nigeria vs Ethiopia,2013-11-16,Nigeria,NG,2013,SUCCESS,Kenneth Omeruo,500000.0,False,Found in Curr Season,False,Kenneth Omeruo,22 Kenneth Omeruo,single,"World Cup Qualifiers, Africa"
3,"World Cup Qualifiers, Africa_0","World Cup Qualifiers, 2014, Africa",Nigeria vs Ethiopia,2013-11-16,Nigeria,NG,2013,SUCCESS,Elderson,3700000.0,False,Found in Curr Season,False,Elderson,3 Elderson,single,"World Cup Qualifiers, Africa"
4,"World Cup Qualifiers, Africa_0","World Cup Qualifiers, 2014, Africa",Nigeria vs Ethiopia,2013-11-16,Nigeria,NG,2013,SUCCESS,Efe Ambrose,400000.0,False,Found in Curr Season,False,Efe Ambrose,5 Efe Ambrose,single,"World Cup Qualifiers, Africa"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9609,"World Cup Qualifiers, Africa_424","World Cup Qualifiers, 2022, Africa",Eritrea vs Namibia,2019-09-04,Namibia,NB,2019,SUCCESS,Willy Stephanus,100000.0,False,Found in Curr Season,False,Willy Stephanus,8 Willy Stephanus,single,"World Cup Qualifiers, Africa"
9610,"World Cup Qualifiers, Africa_424","World Cup Qualifiers, 2022, Africa",Eritrea vs Namibia,2019-09-04,Namibia,NB,2019,SUCCESS,Joslin Kamatuka,75000.0,False,Found in Curr Season,False,Joslin Kamatuka,14 Joslin Kamatuka,single,"World Cup Qualifiers, Africa"
9611,"World Cup Qualifiers, Africa_424","World Cup Qualifiers, 2022, Africa",Eritrea vs Namibia,2019-09-04,Namibia,NB,2019,SUCCESS,Peter Shalulile,600000.0,False,Found in Curr Season,False,Peter Shalulile,13 Peter Shalulile,single,"World Cup Qualifiers, Africa"
9612,"World Cup Qualifiers, Africa_429","World Cup Qualifiers, 2022, Africa",São Tomé and P. vs Guinea-Bissau,2019-09-04,Guinea-Bissau,GW,2019,SUCCESS,Burá Nogueira,125000.0,False,Found in Curr Season,False,Burá Nogueira,6 Burá Nogueira,single,"World Cup Qualifiers, Africa"


In [78]:
afcon_data = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/afcon_lineups_complete.csv')
afcon_data['Competition General'] = 'African Cup of Nations'
#len(afcon_data)
afcon_data = afcon_data.drop(columns={'Unnamed: 0'})
afcon_data

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,Africa Cup of Nations_0,African Cup of Nations 2015,Ivory Coast vs Ghana,2015-02-08,Ivory Coast,CI,2015,SUCCESS,Boubacar Copa,1200000.0,False,Only Before,False,Boubacar Copa,1 Boubacar Copa,single,African Cup of Nations
1,Africa Cup of Nations_0,African Cup of Nations 2015,Ivory Coast vs Ghana,2015-02-08,Ivory Coast,CI,2015,SUCCESS,Wilfried Kanon,500000.0,False,Found in Curr Season,False,Wilfried Kanon,22 Wilfried Kanon,single,African Cup of Nations
2,Africa Cup of Nations_0,African Cup of Nations 2015,Ivory Coast vs Ghana,2015-02-08,Ivory Coast,CI,2015,SUCCESS,Kolo Touré,3600000.0,False,Only Before,False,Kolo Touré,4 Kolo Touré,single,African Cup of Nations
3,Africa Cup of Nations_0,African Cup of Nations 2015,Ivory Coast vs Ghana,2015-02-08,Ivory Coast,CI,2015,SUCCESS,Eric Bailly,4800000.0,False,Used Season After * .8 But Had Both,False,Eric Bailly,21 Eric Bailly,single,African Cup of Nations
4,Africa Cup of Nations_0,African Cup of Nations 2015,Ivory Coast vs Ghana,2015-02-08,Ivory Coast,CI,2015,SUCCESS,Geoffroy Serey Dié,3000000.0,False,Found in Curr Season,False,Geoffroy Serey Dié,20 Geoffroy Serey Dié,single,African Cup of Nations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,Africa Cup of Nations_212,African Cup of Nations 2023,Namibia vs Mali,2024-01-24,Namibia,NB,2024,SUCCESS,Bethuel Muzeu,90000.0,False,Only Before,False,Bethuel Muzeu,9 Bethuel Muzeu,single,African Cup of Nations
4836,Africa Cup of Nations_212,African Cup of Nations 2023,Namibia vs Mali,2024-01-24,Namibia,NB,2024,SUCCESS,Deon Hotto,400000.0,False,Only Before,False,Deon Hotto,7 Deon Hotto,single,African Cup of Nations
4837,Africa Cup of Nations_212,African Cup of Nations 2023,Namibia vs Mali,2024-01-24,Namibia,NB,2024,SUCCESS,Ngero Katua,90000.0,False,Only Before,False,Ngero Katua,6 Ngero Katua,single,African Cup of Nations
4838,Africa Cup of Nations_212,African Cup of Nations 2023,Namibia vs Mali,2024-01-24,Namibia,NB,2024,SUCCESS,Prins Tjiueza,150000.0,False,Only Before,False,Prins Tjiueza,10 Prins Tjiueza,single,African Cup of Nations


In [76]:
gold_cup = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/gold_cup_lineups_complete.csv')
gold_cup['Competition General'] = 'Gold Cup'
#len(gold_cup)
gold_cup = gold_cup.drop(columns={'Unnamed: 0'})
gold_cup

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,Gold Cup_0,Gold Cup 2023,Panama vs Mexico,2023-07-16,Panama,PA,2023,SUCCESS,Orlando Mosquera,350000.0,False,Found in Curr Season,False,Orlando Mosquera,22 Orlando Mosquera,single,Gold Cup
1,Gold Cup_0,Gold Cup 2023,Panama vs Mexico,2023-07-16,Panama,PA,2023,SUCCESS,Andrés Andrade,1500000.0,False,Found in Curr Season,False,Andrés Andrade,16 Andrés Andrade,single,Gold Cup
2,Gold Cup_0,Gold Cup 2023,Panama vs Mexico,2023-07-16,Panama,PA,2023,SUCCESS,Hárold Cummings,380000.0,False,Only Before,False,Hárold Cummings,3 Hárold Cummings,single,Gold Cup
3,Gold Cup_0,Gold Cup 2023,Panama vs Mexico,2023-07-16,Panama,PA,2023,SUCCESS,Fidel Escobar,250000.0,False,Found in Curr Season,False,Fidel Escobar,4 Fidel Escobar,single,Gold Cup
4,Gold Cup_0,Gold Cup 2023,Panama vs Mexico,2023-07-16,Panama,PA,2023,SUCCESS,Aníbal Godoy,300000.0,False,Found in Curr Season,False,Aníbal Godoy,20 Aníbal Godoy,single,Gold Cup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3713,Gold Cup_156,Gold Cup 2013,Martinique vs Mexico,2013-07-14,Martinique,MQ,2013,FAIL - NO DATA,Jordy Delem,0.0,True,Did Not Find in any Season,False,Jordy Delem,15 Jordy Delem,single,Gold Cup
3714,Gold Cup_156,Gold Cup 2013,Martinique vs Mexico,2013-07-14,Martinique,MQ,2013,FAIL - NO DATA,Daniel Hérelle,0.0,True,Did Not Find in any Season,False,Daniel Hérelle,4 Daniel Hérelle,single,Gold Cup
3715,Gold Cup_156,Gold Cup 2013,Martinique vs Mexico,2013-07-14,Martinique,MQ,2013,FAIL - NO DATA,Gaël Germany,0.0,True,Did Not Find in any Season,False,Gaël Germany,5 Gaël Germany,single,Gold Cup
3716,Gold Cup_156,Gold Cup 2013,Martinique vs Mexico,2013-07-14,Martinique,MQ,2013,FAIL - NO DATA,Steeve Gustan,0.0,True,Did Not Find in any Season,False,Steeve Gustan,7 Steeve Gustan,single,Gold Cup


In [74]:
new_asia = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/world_cup_qualifiers_asia_2026_EDITED.csv')
new_asia['Competition General'] = 'World Cup Qualifiers, Asia'
new_asia = new_asia.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})

#len(new_asia)
new_asia

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,"World Cup Qualifiers, Asia_604","World Cup 2026, Qualifiers, Asia",Qatar vs Afghanistan,2023-11-16,Qatar,QA,2023,SUCCESS,Meshaal Barsham,550000.0,False,Found in Curr Season,False,Meshaal Barsham,22 Meshaal Barsham,single,"World Cup Qualifiers, Asia"
1,"World Cup Qualifiers, Asia_604","World Cup 2026, Qualifiers, Asia",Qatar vs Afghanistan,2023-11-16,Qatar,QA,2023,SUCCESS,Boualem Khoukhi,350000.0,False,Found in Curr Season,False,Boualem Khoukhi,16 Boualem Khoukhi,single,"World Cup Qualifiers, Asia"
2,"World Cup Qualifiers, Asia_604","World Cup 2026, Qualifiers, Asia",Qatar vs Afghanistan,2023-11-16,Qatar,QA,2023,SUCCESS,Yousef Aymen,420000.0,False,Only Before,False,Yousef Aymen,17 Yousef Aymen,single,"World Cup Qualifiers, Asia"
3,"World Cup Qualifiers, Asia_604","World Cup 2026, Qualifiers, Asia",Qatar vs Afghanistan,2023-11-16,Qatar,QA,2023,SUCCESS,Homam Al-Amin,500000.0,False,Found in Curr Season,False,Homam Al-Amin,14 Homam Al-Amin,single,"World Cup Qualifiers, Asia"
4,"World Cup Qualifiers, Asia_604","World Cup 2026, Qualifiers, Asia",Qatar vs Afghanistan,2023-11-16,Qatar,QA,2023,SUCCESS,Pedro Miguel,225000.0,False,Found in Curr Season,False,Pedro Miguel,2 Pedro Miguel,single,"World Cup Qualifiers, Asia"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,"World Cup Qualifiers, Asia_719","World Cup 2026, Qualifiers, Asia",Laos vs Nepal,2023-10-17,Nepal,NP,2023,SUCCESS,Dinesh Henjan,0.0,False,Found in Curr Season,False,Dinesh Henjan,15 Dinesh Henjan,single,"World Cup Qualifiers, Asia"
1228,"World Cup Qualifiers, Asia_730","World Cup 2026, Qualifiers, Asia",Afghanistan vs Mongolia,2023-10-12,Afghanistan,AF,2023,SUCCESS,Najim Haidary,0.0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Najim Haidary,12 Najim Haidary,single,"World Cup Qualifiers, Asia"
1229,"World Cup Qualifiers, Asia_730","World Cup 2026, Qualifiers, Asia",Afghanistan vs Mongolia,2023-10-12,Afghanistan,AF,2023,SUCCESS,Omid Popalzay,125000.0,False,Found in Curr Season,False,Omid Popalzay,19 Omid Popalzay,single,"World Cup Qualifiers, Asia"
1230,"World Cup Qualifiers, Asia_730","World Cup 2026, Qualifiers, Asia",Afghanistan vs Mongolia,2023-10-12,Afghanistan,AF,2023,SUCCESS,Noor Husin,0.0,False,Found in Curr Season,False,Noor Husin,6 Noor Husin,single,"World Cup Qualifiers, Asia"


In [99]:
large_combined_df_may_16 = pd.concat([combined_df, wcq_europe, wcq_asia, wcq_africa, new_asia, gold_cup, afcon_data], ignore_index=True)

In [102]:
large_combined_df_may_16['Status'].value_counts()
large_combined_df_may_16.to_csv('large_combined_df_may_16.csv')

In [273]:
(large_combined_df_may_17[large_combined_df_may_17['Competition General'] == 'Olympics'])

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,0,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Mohammed Hameed,100000.0,False,working,False,Mohammed Hameed,Mohammed Hameed,single,Olympics
1,1,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Adnan,1800000.0,False,working,False,Ali Adnan,Ali Adnan,single,Olympics
2,2,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Dhurgham Ismail,1250000.0,False,working,False,Dhurgham Ismail,Dhurgham Ismail,single,Olympics
3,3,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Husni,100000.0,False,working,False,Ali Husni,Ali Hosni,single,Olympics
4,4,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Humam Tariq,100000.0,False,working,False,Humam Tariq,Humam Tariq,single,Olympics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15326,15458,Olympics_1943,"Olympics, 2016",Germany vs South Korea,2016-08-07,South Korea,KR,2016,SUCCESS,Kyu-baek Choi,350000.0,False,Found in Curr Season,False,Kyu-baek Choi,Choi Kyu-baek,single,Olympics
15327,15459,Olympics_1974,"Olympics, 2020",Egypt vs Argentina,2021-07-25,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single,Olympics
15328,15460,Olympics_1977,"Olympics, 2020",Spain vs Argentina,2021-07-28,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single,Olympics
15329,15461,Olympics_1976,"Olympics, 2020",Australia vs Egypt,2021-07-28,Australia,AU,2021,SUCCESS,Dylan Pierias,350000.0,False,Found in Curr Season,False,Dylan Pierias,Dylan Pierias,single,Olympics


In [274]:
large_combined_df_may_17['Competition General'].value_counts()

Competition General
World Cup Qualifiers, Europe           17160
Euros, Qualifiers                      16280
World Cup Qualifiers, Asia             14520
World Cup Qualifiers, Africa           10758
Nations League                         10032
World Cup Qualifiers, CONCACAF          8184
World Cup Qualifiers, CONMEBOL          5522
African Cup of Nations                  4840
World Cup                               4224
Gold Cup                                3718
Copa America                            2464
Euros                                   2244
Olympics                                1408
World Cup Intercontinental Playoffs      220
Name: count, dtype: int64

### Africa 2026

In [126]:
africa_2026_WCQ = load_csv_dataset('africa_2026_wcq_lineups_complete.csv')

In [136]:
africa_2026_WCQ.at[1086, 'Status'] = 'SUCCESS'
africa_2026_WCQ.at[1086, 'Market Value'] = 1200000
africa_2026_WCQ.at[1086, 'Lookup Still Required?'] = False
africa_2026_WCQ.at[1086, 'Lookup Return Case'] = 'Found in Curr Season'

africa_2026_WCQ.at[1114, 'Status'] = 'SUCCESS'
africa_2026_WCQ.at[1114, 'Market Value'] = 0
africa_2026_WCQ.at[1114, 'Lookup Still Required?'] = False
africa_2026_WCQ.at[1114, 'Lookup Return Case'] = 'Found in Curr Season'


In [138]:
africa_2026_WCQ['Status'].value_counts()

Status
SUCCESS    1144
Name: count, dtype: int64

In [131]:
for index, row in africa_2026_WCQ.iterrows():
    if index >= 1114:
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - MANUAL':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == -1:
                    africa_2026_WCQ.at[index, 'Market Value'] = 0
                    africa_2026_WCQ.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    africa_2026_WCQ.at[index, 'Lookup Still Required?'] = False
                    africa_2026_WCQ.at[index, 'Status'] = 'SUCCESS'
                elif test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    africa_2026_WCQ.at[index, 'Market Value'] = test_figure
                    africa_2026_WCQ.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    africa_2026_WCQ.at[index, 'Lookup Still Required?'] = False
                    africa_2026_WCQ.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/mogakolodi-ngele/marktwertverlauf/spieler/154213
passed 1
passed 2
passed 3
{'value': '2013', 'coordinates': (164.1828344291819, 0.0)} 13
{'value': '2015', 'coordinates': (281.24546297381687, 0.0)} 15
{'value': '2017', 'coordinates': (398.46845128358143, 0.0)} 17
{'value': '2019', 'coordinates': (515.5310798282164, 0.0)} 19
{'value': '2021', 'coordinates': (632.754068137981, 0.0)} 21
passed 4
passed 5
passed 6
415 days 00:00:00
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/naby-oulare/marktwertverlauf/spieler/666926
passed 1
passed 2
passed 3
passed 4
passed 5
passed 6
699999.9999999999 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https

In [140]:
africa_2026_WCQ = africa_2026_WCQ.drop(columns={'Unnamed: 0.1', 'Unnamed: 0'})

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,"World Cup Qualifiers, Africa_437","World Cup 2026, Qualifiers, Africa",Ethiopia vs Sierra Leone,2023-11-15,Ethiopia,ET,2023,SUCCESS,Said Habtamu,0.0,False,Only Before,False,Said Habtamu,22 Said Habtamu,single
1,"World Cup Qualifiers, Africa_437","World Cup 2026, Qualifiers, Africa",Ethiopia vs Sierra Leone,2023-11-15,Ethiopia,ET,2023,SUCCESS,Million Solomon,0.0,False,Only Before,False,Million Solomon,5 Million Solomon,single
2,"World Cup Qualifiers, Africa_437","World Cup 2026, Qualifiers, Africa",Ethiopia vs Sierra Leone,2023-11-15,Ethiopia,ET,2023,SUCCESS,Yared Baye,180000.0,False,Only Before,False,Yared Baye,16 Yared Baye,single
3,"World Cup Qualifiers, Africa_437","World Cup 2026, Qualifiers, Africa",Ethiopia vs Sierra Leone,2023-11-15,Ethiopia,ET,2023,SUCCESS,Ramadan Yesuf,120000.0,False,Only Before,False,Ramadan Yesuf,20 Ramadan Yesuf,single
4,"World Cup Qualifiers, Africa_437","World Cup 2026, Qualifiers, Africa",Ethiopia vs Sierra Leone,2023-11-15,Ethiopia,ET,2023,SUCCESS,Henok Adugna,0.0,False,Both Empty,False,Henok Adugna,14 Henok Adugna,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Aprocius Petrus,75000.0,False,Found in Curr Season,False,Aprocius Petrus,18 Aprocius Petrus,single
1140,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Petrus Shitembi,200000.0,False,Found in Curr Season,False,Petrus Shitembi,19 Petrus Shitembi,single
1141,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Deon Hotto,500000.0,False,Found in Curr Season,False,Deon Hotto,7 Deon Hotto,single
1142,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Prins Tjiueza,125000.0,False,Found in Curr Season,False,Prins Tjiueza,10 Prins Tjiueza,single


In [144]:
africa_2026_WCQ['Competition General'] = 'World Cup Qualifiers, Africa'

In [146]:
large_combined_df_may_17 = pd.concat([large_combined_df_may_16, africa_2026_WCQ], ignore_index=True)

In [150]:
large_combined_df_may_17 = large_combined_df_may_17[large_combined_df_may_17['Competition General'] != 'U-20 World Cup']
len(large_combined_df_may_17)

101574

In [152]:
large_combined_df_may_17.to_csv('combined_df_may_17.csv')

In [147]:
large_combined_df_may_17['Competition General'].value_counts()

Competition General
World Cup Qualifiers, Europe           17160
Euros, Qualifiers                      16280
World Cup Qualifiers, Asia             14520
World Cup Qualifiers, Africa           10758
Nations League                         10032
World Cup Qualifiers, CONCACAF          8184
World Cup Qualifiers, CONMEBOL          5522
African Cup of Nations                  4840
World Cup                               4224
Gold Cup                                3718
Copa America                            2464
Euros                                   2244
Olympics                                1408
World Cup Intercontinental Playoffs      220
U-20 World Cup                           132
Name: count, dtype: int64

### U-20 WORLD CUP

In [153]:
u20_data = load_csv_dataset('u20_world_cup_dataset.csv')

In [162]:
u20_data.at[1165, 'Market Value'] = 15000000
u20_data.at[1165, 'Status'] = 'SUCCESS'
u20_data.at[1165, 'Lookup Still Required?'] = False

In [174]:
u20_data.at[1197, 'Market Value'] = 150000
u20_data.at[1197, 'Status'] = 'SUCCESS'
u20_data.at[1197, 'Lookup Still Required?'] = False

In [178]:
u20_data.at[1199, 'Market Value'] = 150000
u20_data.at[1199, 'Status'] = 'SUCCESS'
u20_data.at[1199, 'Lookup Still Required?'] = False

In [220]:
u20_data.to_csv('u20_world_cup_dataset_WORKING.csv')

In [215]:
u20_data[u20_data['Status'] == 'FAIL - MANUAL']['Season'].value_counts()

Season
2023    850
2015    509
2019      4
Name: count, dtype: int64

In [219]:
u20_data['Status'].value_counts()

Status
SUCCESS          1937
FAIL - MANUAL    1359
FAIL - NEW          4
Name: count, dtype: int64

In [213]:
for index, row in u20_data.iterrows():
    if index >= 0:
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - MANUAL':
                if row['Season'] == 2019:
                    name = row["Name(s) Found"]
                    nationality = row['Nationality']
                    date = row['Date']
                    salary_boolean = False
                    test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                    if test_figure == -1:
                        u20_data.at[index, 'Market Value'] = 0
                        u20_data.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                        u20_data.at[index, 'Lookup Still Required?'] = False
                        u20_data.at[index, 'Status'] = 'SUCCESS'
                    elif test_figure == 0:
                        0==0
                        # wcs_africa.at[index, 'Market Value'] = test_figure
                        # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                        # wcs_africa.at[index, 'Lookup Still Required?'] = True
                        # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                    else:
                        u20_data.at[index, 'Market Value'] = test_figure
                        u20_data.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                        u20_data.at[index, 'Lookup Still Required?'] = False
                        u20_data.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/ousseynou-niang/marktwertverlauf/spieler/509463
passed 1
passed 2
passed 3
passed 4
passed 5
passed 6
1132 days 00:00:00
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/ousseynou-niang/marktwertverlauf/spieler/509463
passed 1
passed 2
passed 3
passed 4
passed 5
passed 6
1142 days 00:00:00
finding salary with selenium
link not from transfermarkt. link is from en.wikipedia.org
2
The link is not valid.
Link https://en.wikipedia.org/wiki/Association_football_positions not found in the dataframe.
Kennedy Egbus Mikuni https://www.transfermarkt.us/kennedy-egbus-mikuni/nationalmannschaft/spieler/590648#:~:text=%2320%20Kennedy%20Egbus%20Mikuni&text=Japan%20Position%3A%20Defender%20%2D%20Centre%2D,JEB%20ENTERTAINMENT%20Co.%2CLtd.&text=Kennedy%20Egbus%20Mikuni%20is%20the,Albirex%20Niigata%20(S)).
Kennedy Egbus Mikuni http

### FINAL GROUP

In [5]:
def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = new_leagues_value[(new_leagues_value['PlayerID'] == player_id) & (new_leagues_value['Season'] == year_before)] #leagues_value_large #leagues_value #new_leagues_value
    after_season =  new_leagues_value[(new_leagues_value['PlayerID'] == player_id) & (new_leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before" #add_correct_money
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before" #add_correct_money
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"


def find_in_transfermarkt(player_name, season, country_code, database_name):
    print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        #print('yeeea')
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            #print('der')
            if marketval == '-':
                marketval = 0
            elif '€' in marketval:
                marketval = add_correct_money(marketval)
            marketval = float((marketval)) #add_correct_money
        if marketval != 0:
            #print('wooo')
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            #print(marketval)
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    

In [240]:
new_leagues_value = load_csv_dataset('CSVs we use often/MarketValuesCompleteMay17.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_24576/2775728346.py:54: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [241]:
large_combined_df_may_17 = load_csv_dataset('combined_df_may_17.csv')
for index, row in large_combined_df_may_17.iterrows():
    if row['Name(s) Found'] == 'Martín Payero':
        if row['ORIGINAL JERSEY'] != 'Martín Payero':
            large_combined_df_may_17.at[index, 'Name(s) Found'] = large_combined_df_may_17.at[index, 'ORIGINAL JERSEY']
            large_combined_df_may_17.at[index, 'Name'] = large_combined_df_may_17.at[index, 'ORIGINAL JERSEY']

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_24576/2775728346.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [248]:
#fixing rows with no data
goodcounter = 0
for index, row in large_combined_df_may_17.iterrows():
    if row['Status'] == 'FAIL - NO DATA':
        player_this = row['Name(s) Found']
        full_num = row['Season']
        country_code_this = row['Team Country Code']

        dataset_nationality = new_leagues_value[(new_leagues_value['Team 1 Code'] == country_code_this) & 
            (new_leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            print(index, player_this, len(dataset_nationality))
            goodcounter += 1

            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, new_leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)

            large_combined_df_may_17.at[index, 'Market Value'] = number
            large_combined_df_may_17.at[index, 'Status'] = 'SUCCESS'
            large_combined_df_may_17.at[index, 'Lookup Return Case'] = reason
            large_combined_df_may_17.at[index, 'Lookup Still Required?'] = False
goodcounter
            


90311 Daniel Goitom 17
Daniel Goitom ER 2011
1 0 0
Daniel Goitom -1 Both Empty
90312 Merhawi Kesete 17
Merhawi Kesete ER 2011
1 0 0
Merhawi Kesete -1 Both Empty
90313 Yonatan Solomon 17
Yonatan Solomon ER 2011
1 0 0
Yonatan Solomon -1 Both Empty
90314 Haile Goitom 17
Haile Goitom ER 2011
1 0 0
Haile Goitom -1 Both Empty
90315 Yohannes Tilahun 17
Yohannes Tilahun ER 2011
1 0 0
Yohannes Tilahun -1 Both Empty
90316 Esey Kiflom 17
Esey Kiflom ER 2011
1 0 0
Esey Kiflom -1 Both Empty
90317 Abraham Tedros 17
Abraham Tedros ER 2011
1 0 0
Abraham Tedros -1 Both Empty
90318 Yohannes Nega 17
Yohannes Nega ER 2011
1 0 0
Yohannes Nega -1 Both Empty
90319 Daniel Alexander 17
Daniel Alexander ER 2011
1 0 0
Daniel Alexander -1 Both Empty
90320 Tesfalem Tekle 17
Tesfalem Tekle ER 2011
1 0 0
Tesfalem Tekle -1 Both Empty
90321 Isaias Andeberhan 17
Isaias Andeberhan ER 2011
1 0 0
Isaias Andeberhan -1 Both Empty
90344 Samuel Alazar 17
Samuel Alazar ER 2011
1 0 0
Samuel Alazar -1 Both Empty
90345 Yohannes T

330

In [418]:
large_combined_df_may_17.at[index, 'Market Value'] = 500000
large_combined_df_may_17.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
large_combined_df_may_17.at[index, 'Lookup Still Required?'] = False
large_combined_df_may_17.at[index, 'Status'] = 'SUCCESS'

In [419]:
large_combined_df_may_17.iloc[index]

Unnamed: 0                                  1307
Match ID                           Olympics_2005
Competition                       Olympics, 2020
Match                     Romania vs New Zealand
Date                                  07-28-2021
Nationality                              Romania
Team Country Code                             RO
Season                                      2021
Status                                   SUCCESS
Name                             Ricardo Grigore
Market Value                              500000
Lookup Still Required?                     False
Lookup Return Case          Found in Curr Season
Impute Required?                           False
Name(s) Found                    Ricardo Grigore
ORIGINAL JERSEY                  Ricardo Grigore
Match Case                                single
Competition General                     Olympics
Name: 1243, dtype: object

In [420]:
for index, row in large_combined_df_may_17.iterrows():
    if index >= 1243:
        if row['Status'] == 'FAIL - LAST':
            name = row["Name(s) Found"]
            nationality = row['Nationality']
            date = row['Date']
            salary_boolean = False
            print(index, name)
            test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
            if test_figure == -1:
                large_combined_df_may_17.at[index, 'Market Value'] = 0
                large_combined_df_may_17.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                large_combined_df_may_17.at[index, 'Lookup Still Required?'] = False
                large_combined_df_may_17.at[index, 'Status'] = 'SUCCESS'
            elif test_figure == 0:
                0==0
                # wcs_africa.at[index, 'Market Value'] = test_figure
                # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                # wcs_africa.at[index, 'Lookup Still Required?'] = True
                # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
            else:
                large_combined_df_may_17.at[index, 'Market Value'] = test_figure
                large_combined_df_may_17.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                large_combined_df_may_17.at[index, 'Lookup Still Required?'] = False
                large_combined_df_may_17.at[index, 'Status'] = 'SUCCESS'
        else:
            0==0

1306 Andrei Chindriș
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
Retrying...
Loading... message detected
https://www.transfermarkt.us/andrei-chindris/marktwertverlauf/spieler/435731
passed 1
passed 2
passed 3
{'value': '2016', 'coordinates': (53.95211198428291, 0.0)} 16
{'value': '2017', 'coordinates': (143.87549115913555, 0.0)} 17
{'value': '2018', 'coordinates': (233.79887033398822, 0.0)} 18
{'value': '2019', 'coordinates': (323.7222495088409, 0.0)} 19
{'value': '2020', 'coordinates': (413.89199410609035, 0.0)} 20
{'value': '2021', 'coordinates': (503.81537328094305, 0.0)} 21
{'value': '2022', 'coordinates': (593.7387524557956, 0.0)} 22
passed 4
passed 5
passed 6
1200000.0 12345
1324 Cristian Espinoza
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
Retrying...
Loading... message detected
Retrying...
Loading... message detected
https://www.transfermarkt.us/cristian-espinoza/marktwertverlauf/spieler/278992
pass

In [276]:
large_combined_df_may_17.to_csv('large_combined_df_may_17_WORKING_STILL.csv')

In [283]:
def manual_fix(large_combined_df_may_17, index, name):
    large_combined_df_may_17.at[index, 'Name(s) Found'] = name
    large_combined_df_may_17.at[index, 'Name'] = name
    large_combined_df_may_17.at[index, 'Lookup Still Required?'] = True
    large_combined_df_may_17.at[index, 'Lookup Return Case'] = 'Did Not Find Yet'
    large_combined_df_may_17.at[index, 'Market Value'] = 0
    large_combined_df_may_17.at[index, 'Status'] = 'FAIL - LAST'
    return large_combined_df_may_17

In [368]:
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 29, 'Frederik Børsting')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 58, 'Frederik Børsting')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 306, 'Frederik Børsting')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 36, 'Tebogo Moerane')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 49, 'Gabriel Barbosa')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 47, 'Thiago Maia')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 79, 'Andile Fikizolo')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 164, 'Timo Horn')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 193, 'Timo Horn')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 206, 'Timo Horn')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 292, 'Timo Horn')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 352, 'Timo Horn')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 379, 'Timo Horn')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 272, 'Leandro Vega')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 399, 'Modibo Sagnan')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 400, 'Anthony Caci')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 414, 'Anthony Caci')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 440, 'Anthony Caci')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 534, 'Mohamed Hamdy')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 567, 'Mohamed Hamdy')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 607, 'Mohamed Hamdy')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 776, 'Mohamed Hamdy')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 652, 'Florian Müller')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 687, 'Florian Müller')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 658, 'Marco Richter')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 715, 'Marco Richter')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 715, 'Marco Richter')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1089, 'Ricardo Esgaio')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1091, 'Ricardo Esgaio')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1094, 'Ricardo Esgaio')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1095, 'Ricardo Esgaio')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1090, 'Jonathan Calleri')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1092, 'Jonathan Calleri')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1093, 'Jonathan Calleri')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1150, 'Mikkel Desler')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1152, 'Mikkel Desler')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1154, 'Mikkel Desler')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1165, 'Mikkel Desler')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1161, 'Oussama Methazem')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1168, 'Andrei Sîntean')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1175, 'Abdulbasit Hindi')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1176, 'Abdulbasit Hindi')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1177, 'Abdulbasit Hindi')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1175, 'Abdulbasit Hindi')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1180, 'Ryu Seung-woo')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1181, 'Ryu Seung-woo')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1182, 'Ryu Seung-woo')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1230, 'Thendo Mukumela')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1235, 'Thendo Mukumela')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1234, 'Nkosingiphile Ngcobo')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1243, 'Ricardo Grigore')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1306, 'Andrei Chindriș')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1324, 'Cristian Espinoza')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1330, 'Kim Jin-ya')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1334, 'Kim Jin-ya')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1331, 'Jordan Torunarigha')
large_combined_df_may_17 = manual_fix(large_combined_df_may_17, 1332, 'Kader Keita')




In [ ]:
#the 40 people it didnt find in fail - new we need to do manually



In [424]:
large_combined_df_may_17['Status'].value_counts()

Status
SUCCESS         101370
FAIL - EMPTY       164
FAIL - NEW          40
Name: count, dtype: int64

In [428]:
len(large_combined_df_may_17)

101574

In [427]:
large_combined_df_may_17.to_csv('large_combined_DF_40_REMAINING.csv')

In [438]:
large_combined_df_may_17 = large_combined_df_may_17.drop(columns={'Unnamed: 0'})
large_combined_df_may_17

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Mohammed Hameed,100000.0,False,working,False,Mohammed Hameed,Mohammed Hameed,single,Olympics
1,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Adnan,1800000.0,False,working,False,Ali Adnan,Ali Adnan,single,Olympics
2,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Dhurgham Ismail,1250000.0,False,working,False,Dhurgham Ismail,Dhurgham Ismail,single,Olympics
3,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Husni,100000.0,False,working,False,Ali Husni,Ali Hosni,single,Olympics
4,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Humam Tariq,100000.0,False,working,False,Humam Tariq,Humam Tariq,single,Olympics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101569,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Aprocius Petrus,75000.0,False,Found in Curr Season,False,Aprocius Petrus,18 Aprocius Petrus,single,"World Cup Qualifiers, Africa"
101570,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Petrus Shitembi,200000.0,False,Found in Curr Season,False,Petrus Shitembi,19 Petrus Shitembi,single,"World Cup Qualifiers, Africa"
101571,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Deon Hotto,500000.0,False,Found in Curr Season,False,Deon Hotto,7 Deon Hotto,single,"World Cup Qualifiers, Africa"
101572,"World Cup Qualifiers, Africa_483","World Cup 2026, Qualifiers, Africa",São Tomé and P. vs Namibia,2023-11-21,Namibia,NB,2023,SUCCESS,Prins Tjiueza,125000.0,False,Found in Curr Season,False,Prins Tjiueza,10 Prins Tjiueza,single,"World Cup Qualifiers, Africa"


### AFCON QUALIS

In [435]:
new_afcon_qualis_data = load_csv_dataset('MAY 16 NEW LINEUPS TO ADD TO COMBINED DF/(to add to combined df)/afcon_qualis_lineups_complete_EDITED.csv')
new_afcon_qualis_data['Competition General'] = 'African Cup of Nations, Qualifiers'
new_afcon_qualis_data = new_afcon_qualis_data.drop(columns={'Unnamed: 0', 'Unnamed: 0.1'})
new_afcon_qualis_data

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,"Africa Cup of Nations, Qualifiers_0","African Cup of Nations, 2015 Qualifiers",Sudan vs South Africa,2014-09-05,Sudan,SD,2014,SUCCESS,Elmuez Mahgoub,150000.0,False,Found in Curr Season,False,Elmuez Mahgoub,16 Elmuez Mahgoub,single,"African Cup of Nations, Qualifiers"
1,"Africa Cup of Nations, Qualifiers_0","African Cup of Nations, 2015 Qualifiers",Sudan vs South Africa,2014-09-05,Sudan,SD,2014,SUCCESS,Ali Gaffar,0.0,False,Both Empty,False,Ali Gaffar,7 Ali Gaffar,single,"African Cup of Nations, Qualifiers"
2,"Africa Cup of Nations, Qualifiers_0","African Cup of Nations, 2015 Qualifiers",Sudan vs South Africa,2014-09-05,Sudan,SD,2014,SUCCESS,Amier Kamal,200000.0,False,Found in Curr Season,False,Amier Kamal,18 Amier Kamal,single,"African Cup of Nations, Qualifiers"
3,"Africa Cup of Nations, Qualifiers_0","African Cup of Nations, 2015 Qualifiers",Sudan vs South Africa,2014-09-05,Sudan,SD,2014,SUCCESS,Faris Abdalla,0.0,False,Only After X,False,Faris Abdalla,19 Faris Abdalla,single,"African Cup of Nations, Qualifiers"
4,"Africa Cup of Nations, Qualifiers_0","African Cup of Nations, 2015 Qualifiers",Sudan vs South Africa,2014-09-05,Sudan,SD,2014,SUCCESS,Balla Jabir,100000.0,False,Found in Curr Season,False,Balla Jabir,23 Balla Jabir,single,"African Cup of Nations, Qualifiers"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,"Africa Cup of Nations, Qualifiers_709","African Cup of Nations, 2023 Qualifiers",Eritrea vs Botswana,2022-03-21,Botswana,BW,2022,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
15726,"Africa Cup of Nations, Qualifiers_709","African Cup of Nations, 2023 Qualifiers",Eritrea vs Botswana,2022-03-21,Botswana,BW,2022,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
15727,"Africa Cup of Nations, Qualifiers_709","African Cup of Nations, 2023 Qualifiers",Eritrea vs Botswana,2022-03-21,Botswana,BW,2022,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
15728,"Africa Cup of Nations, Qualifiers_710","African Cup of Nations, 2023 Qualifiers",Seychelles vs Lesotho,2022-03-23,Lesotho,LS,2022,SUCCESS,Rethabile Mokokoane,0.0,True,Found in Curr Season,False,Rethabile Mokokoane,11 Rethabile Mokokoane,single,"African Cup of Nations, Qualifiers"


In [440]:
final_df_may_17 = pd.concat([large_combined_df_may_17, new_afcon_qualis_data], ignore_index=True)
final_df_may_17

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Mohammed Hameed,100000.0,False,working,False,Mohammed Hameed,Mohammed Hameed,single,Olympics
1,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Adnan,1800000.0,False,working,False,Ali Adnan,Ali Adnan,single,Olympics
2,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Dhurgham Ismail,1250000.0,False,working,False,Dhurgham Ismail,Dhurgham Ismail,single,Olympics
3,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Husni,100000.0,False,working,False,Ali Husni,Ali Hosni,single,Olympics
4,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Humam Tariq,100000.0,False,working,False,Humam Tariq,Humam Tariq,single,Olympics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117299,"Africa Cup of Nations, Qualifiers_709","African Cup of Nations, 2023 Qualifiers",Eritrea vs Botswana,2022-03-21,Botswana,BW,2022,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
117300,"Africa Cup of Nations, Qualifiers_709","African Cup of Nations, 2023 Qualifiers",Eritrea vs Botswana,2022-03-21,Botswana,BW,2022,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
117301,"Africa Cup of Nations, Qualifiers_709","African Cup of Nations, 2023 Qualifiers",Eritrea vs Botswana,2022-03-21,Botswana,BW,2022,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
117302,"Africa Cup of Nations, Qualifiers_710","African Cup of Nations, 2023 Qualifiers",Seychelles vs Lesotho,2022-03-23,Lesotho,LS,2022,SUCCESS,Rethabile Mokokoane,0.0,True,Found in Curr Season,False,Rethabile Mokokoane,11 Rethabile Mokokoane,single,"African Cup of Nations, Qualifiers"


In [473]:
final_df_may_17.to_csv('final_df_may_17_EDITED_40_REMAINING.csv')

In [456]:
pd.set_option('display.max_rows', 50)


### FIX THE OLYMPICS MATCH IDS

In [494]:
new_combined_Df = load_csv_dataset('final_df_may_17_EDITED_40_REMAINING.csv')
new_combined_Df = new_combined_Df.drop(columns={'Unnamed: 0'})

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_24576/2775728346.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [495]:
def convert_date_format(date_str):
    try:
        # Check if the date is in MM-DD-YYYY format
        if '-' in date_str and len(date_str) == 10:
            if len(date_str.split('-')[0]) == 4:
                return date_str
            else:
                month, day, year = date_str.split('-')
                # Convert and return in YYYY-MM-DD format
                return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
        else:
            # Return the date as is if it doesn't match the expected format
            return date_str
    except:
        # Return the date as is in case of any unexpected issues
        return date_str

# Apply the date conversion conditionally
new_combined_Df.loc[new_combined_Df['Competition General'] == 'Olympics', 'Date'] = \
    new_combined_Df.loc[new_combined_Df['Competition General'] == 'Olympics', 'Date'].apply(convert_date_format)

In [496]:
new_combined_Df.loc[new_combined_Df['Competition General'] == 'Olympics']

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
0,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Mohammed Hameed,100000.0,False,working,False,Mohammed Hameed,Mohammed Hameed,single,Olympics
1,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Adnan,1800000.0,False,working,False,Ali Adnan,Ali Adnan,single,Olympics
2,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Dhurgham Ismail,1250000.0,False,working,False,Dhurgham Ismail,Dhurgham Ismail,single,Olympics
3,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Ali Husni,100000.0,False,working,False,Ali Husni,Ali Hosni,single,Olympics
4,Olympics_1928,"Olympics, 2016",Iraq vs Denmark,2016-08-04,Iraq,IQ,2016,SUCCESS,Humam Tariq,100000.0,False,working,False,Humam Tariq,Humam Tariq,single,Olympics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15326,Olympics_1943,"Olympics, 2016",Germany vs South Korea,2016-08-07,South Korea,KR,2016,SUCCESS,Kyu-baek Choi,350000.0,False,Found in Curr Season,False,Kyu-baek Choi,Choi Kyu-baek,single,Olympics
15327,Olympics_1974,"Olympics, 2020",Egypt vs Argentina,2021-07-25,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single,Olympics
15328,Olympics_1977,"Olympics, 2020",Spain vs Argentina,2021-07-28,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single,Olympics
15329,Olympics_1976,"Olympics, 2020",Australia vs Egypt,2021-07-28,Australia,AU,2021,SUCCESS,Dylan Pierias,350000.0,False,Found in Curr Season,False,Dylan Pierias,Dylan Pierias,single,Olympics


In [ ]:
#fix the olympics match ID system


#for each row
#if its an olympics match

#and the format is MM-DD-YYYY 
#change it to YYYY-MM-DD

In [ ]:
#now that i have all of the olympics matches and dates grouped together in 22's how do i use that? 

In [530]:
olympicdf = new_combined_Df[new_combined_Df['Competition General'] == 'Olympics']#['Match ID'].unique()

grouped_df = olympicdf.groupby(['Match', 'Date']).size().reset_index(name='Count')
grouped_df[grouped_df['Count'] != 22]
# If you want to perform an aggregate function, you can add it, e.g., sum#.head(50)


#make a column in the grouped DF which is the Match ID. then pull it from there 

,Match,Date,Count


In [532]:
grouped_df['Match ID'] = 'Olympics' + grouped_df.index.astype(str)

grouped_df

,Match,Date,Count,Match ID
0,Algeria vs Portugal,2016-08-10,22,Olympics0
1,Argentina vs Algeria,2016-08-07,22,Olympics1
2,Argentina vs Australia,2021-07-22,22,Olympics2
3,Argentina vs Honduras,2016-08-10,22,Olympics3
4,Australia vs Egypt,2021-07-28,22,Olympics4
...,...,...,...,...
59,South Korea vs Mexico,2021-07-31,22,Olympics59
60,Spain vs Argentina,2021-07-28,22,Olympics60
61,Spain vs Ivory Coast,2021-07-31,22,Olympics61
62,Sweden vs Colombia,2016-08-04,22,Olympics62


In [541]:
cunter = 0
for index, row in new_combined_Df.iterrows():
    if row['Competition General'] == 'Olympics':
        cunter += 1
        match = row['Match']
        date = row['Date']
        grouped_DF_row = grouped_df[(grouped_df['Match'] == match) & (grouped_df['Date'] == date)]['Match ID'].unique()
        if len(grouped_DF_row) == 1:
            correct_match_ID = grouped_DF_row[0]
            new_combined_Df.at[index, 'Match ID'] = correct_match_ID

In [547]:
match_ID_DF = new_combined_Df.groupby(['Match ID']).size().reset_index(name='Count')
match_ID_DF[match_ID_DF['Count'] != 22]

,Match ID,Count


In [546]:
new_combined_Df.to_csv('may_17_combined_DF_40_REMAINING.csv')

In [551]:
new_combined_Df['Status'].value_counts()

Status
SUCCESS         116931
FAIL - EMPTY       333
FAIL - NEW          40
Name: count, dtype: int64

## May 20 - on rows i fixed manually


In [9]:
leagues_value = load_csv_dataset('CSVs we use often/MarketValuesCompleteMay17.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_19354/2775728346.py:54: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [13]:
latest_combined_DF = load_csv_dataset('combined_DF_may_20.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_19354/2775728346.py:54: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [14]:
for index, row in latest_combined_DF.iterrows():
    if index >= 0:
        if row['Status'] == 'POSSIBLE':
            name = row["ORIGINAL JERSEY"]
            nationality = row['Nationality']
            date = row['Date']
            salary_boolean = False
            print(index, name)
            test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
            if test_figure == -1:
                latest_combined_DF.at[index, 'Market Value'] = 0
                latest_combined_DF.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                latest_combined_DF.at[index, 'Lookup Still Required?'] = False
                latest_combined_DF.at[index, 'Status'] = 'SUCCESS'
            elif test_figure == 0:
                0==0
                # wcs_africa.at[index, 'Market Value'] = test_figure
                # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                # wcs_africa.at[index, 'Lookup Still Required?'] = True
                # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
            else:
                latest_combined_DF.at[index, 'Market Value'] = test_figure
                latest_combined_DF.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                latest_combined_DF.at[index, 'Lookup Still Required?'] = False
                latest_combined_DF.at[index, 'Status'] = 'SUCCESS'
        else:
            0==0

0 Sayed Mohammed Jaffer
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
1 Dawood Saad
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
2 Husain Ali Baba
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
3 Saleh Abdulhameed
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
4 Mohammed Husain
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
5 Faouzi Aaish
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
6 Hamad Rakea
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem c

In [19]:
latest_combined_DF = latest_combined_DF[latest_combined_DF['Match ID'] != 'Africa Cup of Nations, Qualifiers_258']
latest_combined_DF = latest_combined_DF.drop(columns={'Unnamed: 0.1', 'Unnamed: 0'})
latest_combined_DF['Match ID'].value_counts()

Match ID
World Cup Qualifiers, Asia_95            22
World Cup Qualifiers, Asia_453           22
World Cup Qualifiers, Asia_451           22
World Cup Qualifiers, Asia_450           22
World Cup Qualifiers, Asia_449           22
                                         ..
Copa America_9                           22
Copa America_8                           22
Copa America_7                           22
Copa America_6                           22
Africa Cup of Nations, Qualifiers_714    22
Name: count, Length: 5319, dtype: int64

In [31]:
latest_combined_DF.at[24, 'Market Value'] = 125000
latest_combined_DF.at[24, 'Lookup Return Case'] = 'Found in Curr Season'
latest_combined_DF.at[24, 'Lookup Still Required?'] = False
latest_combined_DF.at[24, 'Status'] = 'SUCCESS'


latest_combined_DF.at[26, 'Market Value'] = 150000
latest_combined_DF.at[26, 'Lookup Return Case'] = 'Found in Curr Season'
latest_combined_DF.at[26, 'Lookup Still Required?'] = False
latest_combined_DF.at[26, 'Status'] = 'SUCCESS'


latest_combined_DF.at[47, 'Market Value'] = 50000
latest_combined_DF.at[47, 'Lookup Return Case'] = 'Found in Curr Season'
latest_combined_DF.at[47, 'Lookup Still Required?'] = False
latest_combined_DF.at[47, 'Status'] = 'SUCCESS'


In [33]:
for index, row in latest_combined_DF.iterrows():
    if row['Status'] == 'POSSIBLE':
        latest_combined_DF.at[index, 'Status'] = 'SUCCESS'
        latest_combined_DF.at[index, 'Market Value'] = 0
        latest_combined_DF.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
        latest_combined_DF.at[index, 'Lookup Still Required?'] = False


In [40]:
latest_combined_DF[latest_combined_DF['Status'] == 'FAIL - EMPTY']

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General
28,"World Cup Qualifiers, Africa_111","World Cup Qualifiers, 2014, Africa",DR Congo vs Libya,2013-03-24,Libya,LY,2013,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Africa"
29,"World Cup Qualifiers, Africa_111","World Cup Qualifiers, 2014, Africa",DR Congo vs Libya,2013-03-24,Libya,LY,2013,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Africa"
30,"World Cup Qualifiers, Africa_111","World Cup Qualifiers, 2014, Africa",DR Congo vs Libya,2013-03-24,Libya,LY,2013,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Africa"
31,"World Cup Qualifiers, Africa_111","World Cup Qualifiers, 2014, Africa",DR Congo vs Libya,2013-03-24,Libya,LY,2013,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"World Cup Qualifiers, Africa"
39,"Africa Cup of Nations, Qualifiers_159","African Cup of Nations, 2017 Qualifiers",Benin vs Equat. Guinea,2016-06-12,Benin,BJ,2016,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
40,"Africa Cup of Nations, Qualifiers_159","African Cup of Nations, 2017 Qualifiers",Benin vs Equat. Guinea,2016-06-12,Equatorial Guinea,GQ,2016,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
43,"Africa Cup of Nations, Qualifiers_188","African Cup of Nations, 2017 Qualifiers",São Tomé and P. vs Morocco,2015-09-05,São Tomé and Príncipe,ST,2015,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
44,"Africa Cup of Nations, Qualifiers_188","African Cup of Nations, 2017 Qualifiers",São Tomé and P. vs Morocco,2015-09-05,São Tomé and Príncipe,ST,2015,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
45,"Africa Cup of Nations, Qualifiers_219","African Cup of Nations, 2017 Qualifiers",Sudan vs Sierra Leone,2015-06-14,Sierra Leone,SL,2015,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
46,"Africa Cup of Nations, Qualifiers_219","African Cup of Nations, 2017 Qualifiers",Sudan vs Sierra Leone,2015-06-14,Sierra Leone,SL,2015,FAIL - EMPTY,NaN,0.0,True,Did Not Find in any Season,False,NaN,-,single,"African Cup of Nations, Qualifiers"
